# Functions for making the dataframes for the VADs

In [44]:
import pyart
import numpy as np
from datetime import datetime, timedelta
import glob
# from radarcalc import *
import matplotlib.pyplot as plt
import pandas as pd
import metpy.calc as mpcalc
import metpy
import metpy.plots
from metpy.units import units
import cartopy.crs as ccrs
import gc
from astropy.convolution import convolve
from boto.s3.connection import S3Connection
import tempfile
import copy
import matplotlib
import xarray as xr
import math
from datetime import datetime

In [45]:
def dealias_Ka(radar,PPIGC_flag=False):
    '''
    This function aims to take care of all the nitty gritty customizations of pyarts dealiasing specifically for TTUKa radar data.

    Parameter: radar (pyart object), PPIGC_flag (boolean *kwarg)
    Returns: radar (pyart object) with corrected_velocity field and velocity_texture field added on

    How it works:
    1. Calculates velocity texture and creates a histogram based on all of the magnitudes of textures at each bin for the entire volume
    2. loops through each indivual sweep
    3. infinite loop to iterate between the minimum amount of texture between textures 1 and 6 and the maximum amount of texture in the histogram, with 0.5 m/s steps
    4. Each iteration, create a gatefilter that filters the bins above that texture value and dealias it with the gatefilter to ignore the high textured regions
    5. If the maximum texture is reached before breaking, set the gatefilter to mask textures above 12 m/s
    6. if the scan is an RHI, run a 2 pass variance filter along each ray.
        This convolves a 71 sized boxcar with the data, and if the difference between the point the boxcar is centered on and the mean of the boxcar is greater than the nyquist, then add/subtract 2*nyquist to that point
    7. Then, take the difference between the mean of the bottom 4 rays of the dealiased velocity in the RHI and the bottom 4 rays of the aliased velocity
    8. If the absolute difference is larger than the nyquist, the either subtract 2* nyquist or add 2*nyquist to the entire sweep and break the loop
    9. If the absolute difference is less than the nyquist, no fixes need to be applied, break the loop
    10. If the scan is a PPI, do steps 8 and 9, but skip steps 6 and 7
    11. Outside the infinite loop, add the option of the PPIGC_flag, a boolean flag that will help maintain ground clutter in PPIs at 0 m/s
        This works by identifying regions of very low spectrum width (<0.1 m/s), and setting the velocity in those regions = 0. Please note, this may introduce artificial speckles of 0 in real data, where spectrum width is noisy
    12. Apply the alias fix algorithm which convolves a boxcar of specified boxcar of size 9 with the data, and if the variance between the middle pixel of interest and the mean is greater than the nyquist, then flip it back over
    12. At the end of each sweep, assign the data from that processed sweep into a dictionary, then add the dictionary as the corrected_velocity field.
    
    The aforementioned method is FAR from perfect, but is as robust as I can do currently. One thing to improve this though is to use the UNRAVEL algorithm: https://github.com/vlouf/dealias
    The UNRAVEL algorithm shows remarkable error characteristics compared to "competitors", possibly at a time cost, which isn't a HUGE deal for us. Downside is it may not work for our "volumes" since they are temporally uncorrelated and are not full volumes through the atmosphere
    '''
    
    vel_texture = pyart.retrieve.calculate_velocity_texture(radar, vel_field='velocity', wind_size=3)
    radar.add_field('velocity_texture', vel_texture, replace_existing=True)
    hist, bins = np.histogram(radar.fields['velocity_texture']['data'][~np.isnan(radar.fields['velocity_texture']['data'])], bins=150)
    bins = (bins[1:]+bins[:-1])/2.0
    gatefilter = pyart.filters.GateFilter(radar)
    velocity_dealiased = pyart.correct.dealias_region_based(radar, vel_field='velocity', nyquist_vel=radar.instrument_parameters['nyquist_velocity']['data'][0], centered=True) #standin, data will be replaced

    for swp_id in range(radar.nsweeps):
        #get indices from beginning and ending of sweep
        sw_start = radar.sweep_start_ray_index['data'][swp_id]
        sw_end = radar.sweep_end_ray_index['data'][swp_id]+1
        # if the scan is a gentle scan we skip it
        if ((radar.nsweeps==1) & (np.around(radar.fixed_angle['data'][0],decimals=0)==37.0) & (np.abs(np.mean(radar.fields['velocity']['data']))<0.1) & (np.std(radar.fields['velocity']['data'])<1)):
            continue
            
        else:

            counter = 0
            while True: #do an infinite loop and either break it when the data is unfolded correctly or when the max texture is reached
                #if the bin with the lowest count between textures 1 and 6 + i*0.5 is less than the maximum amount of bins
                if bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]]+counter*0.5 < np.amax(bins):
                    gatefilter.exclude_above('velocity_texture', bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]]+counter*0.5)
                    nyq = radar.instrument_parameters['nyquist_velocity']['data'][0]
                    vede = pyart.correct.dealias_region_based(radar, vel_field='velocity', nyquist_vel=nyq,
                                                                            centered=True, gatefilter=gatefilter)
                else:
                    gatefilter.exclude_above('velocity_texture', 12)#bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]])
                    nyq = radar.instrument_parameters['nyquist_velocity']['data'][0]
                    vede = pyart.correct.dealias_region_based(radar, vel_field='velocity', nyquist_vel=nyq,
                                                                            centered=True, gatefilter=gatefilter)
    
    
                np.ma.set_fill_value(vede['data'], np.nan)
                #extract mask so we can apply the correct gatefilters on later
                mask=np.ma.getmask(vede['data'])
    
                #apply mask to velocity field and fix the small blips from dealiasing
                if radar.scan_type == 'rhi':
                    #pass 1 of variance filtering along the ray.
                    #Convolves a 71 sized boxcar with the data, and if the difference between the point the boxcar is centered on and the mean of the boxcar is greater than the nyquist, then add/subtract 2*nyquist to that point
                    vel = vede['data'].filled()
    
                    for sw in range(np.shape(vel)[0]):
                        mean = convolve(vel[sw,:],np.ones(71))
                        var = vel[sw,:]-mean
                        high_idx = var > nyq
                        low_idx = var < -nyq
                        vel[sw,:][high_idx] = vel[sw,:][high_idx] - 2*nyq
                        vel[sw,:][low_idx] = vel[sw,:][low_idx] + 2*nyq
                    vede['data']=np.ma.masked_array(vel,mask=mask,fill_value=np.nan)
    
                    #pass 2 of variance filtering along the ray. In case there are errant folds than need to be folded back
                    vel = vede['data'].filled()
                    for sw in range(np.shape(vel)[0]):
                        mean = convolve(vel[sw,:],np.ones(71))
                        var = vel[sw,:]-mean
                        high_idx = var > nyq
                        low_idx = var < -nyq
                        vel[sw,:][high_idx] = vel[sw,:][high_idx] - 2*nyq
                        vel[sw,:][low_idx] = vel[sw,:][low_idx] + 2*nyq
                    vede['data']=np.ma.masked_array(vel,mask=mask,fill_value=np.nan)

                    #find means of the bottom 4 rays of the RHI(should be close to 0) and compare the dealiased velocities to the aliased velocities
                    np.ma.set_fill_value(radar.fields['velocity']['data'], np.nan)
                    meanvelal = np.mean(radar.fields['velocity']['data'][sw_start:sw_start+4,:].filled()[~np.isnan(radar.fields['velocity']['data'][sw_start:sw_start+4,:].filled())])
                    meanveldeal = np.mean(vede['data'][sw_start:sw_start+4,:].filled()[~np.isnan(vede['data'][sw_start:sw_start+4,:].filled())])
                    if np.abs(meanvelal-meanveldeal) < nyq: #nyq is an arbitrary threshold and should be tuned
                        break
                    if bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]]+counter*0.5 < np.amax(bins):
                        if (meanvelal-meanveldeal) > 0:
                            vede['data'][sw_start:sw_end,:] += 2*nyq
                        else:
                            vede['data'][sw_start:sw_end,:] -= 2*nyq
                        break
                if radar.scan_type == 'ppi':
                    np.ma.set_fill_value(radar.fields['velocity']['data'], np.nan)
                    meanvelal = np.mean(radar.fields['velocity']['data'][sw_start:sw_end,:].filled()[~np.isnan(radar.fields['velocity']['data'][sw_start:sw_end,:].filled())])
                    meanveldeal = np.mean(vede['data'][sw_start:sw_end,:].filled()[~np.isnan(vede['data'][sw_start:sw_end,:].filled())])
                    if np.abs(meanvelal-meanveldeal) < nyq: #nyq is an arbitrary threshold and should be tuned
                        break
                    if bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]]+counter*0.5 < np.amax(bins):
                        if (meanvelal-meanveldeal) > 0:
                            vede['data'][sw_start:sw_end,:] += 2*nyq
                        else:
                            vede['data'][sw_start:sw_end,:] -= 2*nyq
                        break
                counter+=1
            
        #put alias fix inside here instead of calling it to make it more portable
        delta=3
        mean = convolve(vede['data'][sw_start:sw_end,:],np.ones((delta,delta))/delta**2.)
        mean[0,:] = vede['data'][sw_start:sw_end,:][0,:]
        mean[-1,:] = vede['data'][sw_start:sw_end,:][-1,:]
        var = vede['data'][sw_start:sw_end,:] - mean

        high_idx = np.logical_and(var > nyq, var < 4*nyq)
        low_idx = np.logical_and(var < -nyq, var > -4*nyq)

        vede['data'][sw_start:sw_end,:][high_idx] = vede['data'][sw_start:sw_end,:][high_idx] - 2*nyq
        vede['data'][sw_start:sw_end,:][low_idx] = vede['data'][sw_start:sw_end,:][low_idx] + 2*nyq

        #corrects ground clutter by arbitrarily setting the velocity equal to 0 where spectrum width is less than 0.075 m/s
        if PPIGC_flag == True:
            if radar.scan_type == 'ppi':
                sw = radar.fields['spectrum_width']['data'][sw_start:sw_end,:].filled()
                vel = radar.fields['velocity']['data'][sw_start:sw_end,:].filled()
                mask = sw<0.1
                vede['data'][sw_start:sw_end,:] = np.where(~mask,vede['data'][sw_start:sw_end,:],0)

        velocity_dealiased['data'][sw_start:sw_end,:] = vede['data'][sw_start:sw_end,:]
        velocity_dealiased['data'][sw_start:sw_end,:] = alias_fix(velocity_dealiased['data'][sw_start:sw_end,:],nyq,delta=9)
    radar.add_field('corrected_velocity', velocity_dealiased, replace_existing=True)

    return radar

def alias_fix(vel,nyq,delta=3):
    '''
    !!!!!!!!!!!!!!!!!!
    Removes dealiasing errors around the periphery of a folded region

    Parameters: velocity array (array), nyquist velocity (number), size of window (int, must be odd, unity is no change)
    Returns: cleaned velocity array (array)
    '''
    mean = convolve(vel,np.ones((delta,delta))/delta**2.)
    mean[0,:] = vel[0,:]
    mean[-1,:] = vel[-1,:]
    var = vel - mean

    high_idx = np.logical_and(var > nyq, var < 4*nyq)
    low_idx = np.logical_and(var < -nyq, var > -4*nyq)

    vel[high_idx] = vel[high_idx] - 2*nyq
    vel[low_idx] = vel[low_idx] + 2*nyq

    return vel

def get_radar_from_aws(site, datetime_t, datetime_te):
    """
    Get the closest volume of NEXRAD data to a particular datetime.
    Parameters
    ----------
    site : string
        four letter radar designation
    datetime_t : datetime
        desired date time
    Returns
    -------
    radar : Py-ART Radar Object
        Radar closest to the queried datetime
    """

    # First create the query string for the bucket knowing
    # how NOAA and AWS store the data
    my_pref = datetime_t.strftime('%Y/%m/%d/') + site

    # Connect to the bucket
    conn = S3Connection(anon = True)
    bucket = conn.get_bucket('noaa-nexrad-level2')

    # Get a list of files
    bucket_list = list(bucket.list(prefix = my_pref))

    # we are going to create a list of keys and datetimes to allow easy searching
    keys = []
    datetimes = []

    # populate the list
    for i in range(len(bucket_list)):
        this_str = str(bucket_list[i].key)
        if 'gz' in this_str:
            endme = this_str[-22:-4]
            fmt = '%Y%m%d_%H%M%S_V0'
            dt = datetime.strptime(endme, fmt)
            datetimes.append(dt)
            keys.append(bucket_list[i])

        if this_str[-3::] == 'V06':
            endme = this_str[-19::]
            fmt = '%Y%m%d_%H%M%S_V06'
            dt = datetime.strptime(endme, fmt)
            datetimes.append(dt)
            keys.append(bucket_list[i])

    # find the closest available radar to your datetime
    closest_datetime_b = _nearestDate(datetimes, datetime_t)
    closest_datetime_e = _nearestDate(datetimes, datetime_te)

    index_b = datetimes.index(closest_datetime_b)
    index_e = datetimes.index(closest_datetime_e)

    radar_namelist = keys[index_b:index_e+1]
    radar_list=[]
    for i in range(np.shape(radar_namelist)[0]):
        localfile = tempfile.NamedTemporaryFile()
        radar_namelist[i].get_contents_to_filename(localfile.name)
        radar_list.append(pyart.io.read(localfile.name))
    return radar_namelist,radar_list

def getLocation(lat1, lon1, brng, distancekm):
    lat1 = lat1 * np.pi / 180.0
    lon1 = lon1 * np.pi / 180.0
    #earth radius
    R = 6378.1
    #R = ~ 3959 MilesR = 3959
    bearing = (brng / 90.)* np.pi / 2.

    lat2 = np.arcsin(np.sin(lat1) * np.cos(distancekm/R) + np.cos(lat1) * np.sin(distancekm/R) * np.cos(bearing))
    lon2 = lon1 + np.arctan2(np.sin(bearing)*np.sin(distancekm/R)*np.cos(lat1),np.cos(distancekm/R)-np.sin(lat1)*np.sin(lat2))
    lon2 = 180.0 * lon2 / np.pi
    lat2 = 180.0 * lat2 / np.pi
    return lat2, lon2

def _nearestDate(dates, pivot):
    return min(dates, key=lambda x: abs(x - pivot))

def find_nearest(array, value):
    '''
    Function to find index of the array in which the value is closest to

    Parameters: array (array), value (number)
    Returns: index (int)

    Example: xind = CM1calc.find_nearest(x,5)
    '''

    array = np.asarray(array)
    idx = (np.abs(array-value)).argmin()
    return idx


def vehicle_correction_vad(radar,df):
    '''
    Function that creates a 'vad_corrected_velocity' field that can be used for vad calculations, 
    but should be general enough to use for stationary VADs as well as moving PPIs. 
    Other than adding the new field, the radar times are smoothly interpolated and the azimuths are 
    corrected via the GPS pandas dataframe.
    
    Parameters: pyart radar object (object), pandas dataframe of appropriate radarGPS file (dataframe)
    Returns: pyart radar object (object), speed (float), speed variance (float), bearing (float), bearing variance (float), 
             latitude (float), latitude variance (float), longitude (float), longitude variance (float)
    
    Example: radar, velmean, velvar, bearmean, bearvar, latmean, latvar, lonmean, lonvar = vehicle_correction_vad(radar,df)

    p.s. only works if the velocity is already dealiased and there is a 'corrected_velocity' field
         also only works if a single sweep is extracted, example: radar = radar.extract_sweeps([0])
    '''
    #for i in df:
    #orders the time to increase monotonically instead of having a massive step jump in the middle
    roll_mag = (np.argmax(np.abs(np.gradient(radar.time['data'])))+1)
    times = np.roll(radar.time['data'],-roll_mag) 
    
    #a complicated way to create linear increasing times (instead of steps) that start at 0 seconds after the time datum and increase to the middle of the second max time plateau (if confused, plotting it is helpful)
    #from now on, we are going to assume ray_times is the fractional seconds after the time datum the ray is gathered, and we need to roll it back to match with the rest of the data
    ray_times = np.roll(np.arange(0,((np.unique(times)[-2])/(find_nearest(times,np.unique(times)[-2])+int(np.sum(radar.time['data']==np.unique(times)[-2])/2)))*len(times)+1e-11,((np.unique(times)[-2])/(find_nearest(times,np.unique(times)[-2])+int(np.sum(radar.time['data']==np.unique(times)[-2])/2)))),roll_mag)

    radar.time['data']=ray_times
    
    #df['datetime'] = pd.to_datetime(df['ddmmyy']+df['hhmmss[UTC]'], format='%d%m%y%H%M%S')
    df['datetime'] = [datetime.strptime(d,'%d%m%y%H%M%S') for d in df['ddmmyy']+[f'{h:06}' for h in df['hhmmss[UTC]'].astype(int)]]
    beginscanindex = df.loc[df['datetime'] == datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')].index
    endscanindex = df.loc[df['datetime'] == datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')].index+np.ceil(np.amax(ray_times))+1
    if len(beginscanindex) == 0: # MAKES IT SO THAT IF THE DATETIME IS MISSING IT SKIPS THE FILE
        return None#, None, None, None, None, None, None, None
    dfscan = df.iloc[beginscanindex[0].astype(int):endscanindex[0].astype(int)]
    dfscan = dfscan.astype({'Bearing[degrees]': 'float'})
    dfscan = dfscan.astype({'Velocity[knots]': 'float'})
        
    ray_bearings = np.interp(ray_times,np.arange(len(dfscan)),dfscan['Bearing[degrees]'])
    ray_speeds = np.interp(ray_times,np.arange(len(dfscan)),dfscan['Velocity[knots]'])
        
    #    print('velocity [kts]',dfscan['Velocity[knots]'].mean(),'+-',dfscan['Velocity[knots]'].var())
    speed = dfscan['Velocity[knots]'].mean()
    #    print('bearing',dfscan['Bearing[degrees]'].mean(),'+-',dfscan['Bearing[degrees]'].var())
    bearing = dfscan['Bearing[degrees]'].mean()
    #    print('latitude',dfscan['Latitude'].astype(float).mean(),'+-',dfscan['Latitude'].astype(float).var())
    lat = dfscan['Latitude'].astype(float).mean()
    #    print('longitude',dfscan['Longitude'].astype(float).mean(),'+-',dfscan['Longitude'].astype(float).var())
    lon = dfscan['Longitude'].astype(float).mean()
        
    radar.azimuth['data'] += ray_bearings[:-1] #bearing
        
    rad_vel = copy.deepcopy(radar.fields['corrected_velocity'])
        
    rad_vel['data']+=(np.cos(np.deg2rad(radar.azimuth['data']-ray_bearings[:-1]))*(ray_speeds[:-1]/1.94384)*np.cos(np.deg2rad(radar.fixed_angle['data'][0])))[:,np.newaxis]
        
    #fix mask, remove points very close to radar as well as the very last bin, more often than not, = bad data
    rad_vel['data'].mask[:,:5] = True
    rad_vel['data'].mask[:,-1] = True
    radar.add_field('vad_corrected_velocity', rad_vel, replace_existing=True)
        
    return radar, dfscan['Velocity[knots]'].mean(),dfscan['Velocity[knots]'].var(),dfscan['Bearing[degrees]'].mean(),dfscan['Bearing[degrees]'].var(),dfscan['Latitude'].astype(float).mean(),dfscan['Latitude'].astype(float).var(),dfscan['Longitude'].astype(float).mean(),dfscan['Longitude'].astype(float).var()
                

In [46]:
# from stackoverflow (https://stackoverflow.com/questions/639695/how-to-convert-latitude-or-longitude-to-meters)
# originally in javascript
def measure(lat1, lon1, lat2, lon2):  # generally used geo measurement function
    R = 6378.137 # Radius of earth in KM
    dLat = lat2 * np.pi / 180 - lat1 * np.pi / 180;
    dLon = lon2 * np.pi / 180 - lon1 * np.pi / 180;
    a = np.sin(dLat/2) * np.sin(dLat/2) + np.cos(lat1 * np.pi / 180) * np.cos(lat2 * np.pi / 180) * np.sin(dLon/2) * np.sin(dLon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = R * c
    #d_meters = d * 1000
    return d # km

In [47]:
def get_storm_tobac(tobac_file, cell_number):
    '''
    This function will use the data created by using tobac to obtain the 
    information associated with a desired cell.
    Returns the latitudes, longitudes, and times associated with the desired cell.
    '''
    tobac_features_xr = xr.open_dataset(tobac_file)
    idx = tobac_features_xr['idx'].data
    cell = tobac_features_xr['cell'].data
    morton_storm_indeces_idx = np.where(idx == cell_number)
    morton_storm_indeces = np.where(cell == cell_number)
    tobac_times = tobac_features_xr['time']
    tobac_lats = np.array(tobac_features_xr['latitude'])
    tobac_lons = np.array(tobac_features_xr['longitude'])
    morton_tobac_lats = tobac_lats[morton_storm_indeces]
    morton_tobac_lons = tobac_lons[morton_storm_indeces]
    morton_tobac_times = tobac_times[morton_storm_indeces]
    morton_cell_29 = cell[morton_storm_indeces]
    morton_tobac_times_datetime = morton_tobac_times.astype('datetime64[s]')
    
    return morton_tobac_lats, morton_tobac_lons, morton_cell_29, morton_tobac_times_datetime

In [48]:
def nearest_tobac_time(netcdf_list, morton_tobac_lats, morton_tobac_lons, morton_tobac_times_datetime):
    '''
    To run this function, you must make sure you've pre-defined the generic find_nearest function first.
    netcdf_file = a single netcdf file
    morton_tobac_times_datetime = the list made from the get_storm_tobac function
    '''
    time_list = []
    for netcdf_file in netcdf_list:
        time_yoink = netcdf_file[-15:-3]
        #print(time_yoink)
        time_yoink_dt = datetime.strptime(time_yoink, '%y%m%d%H%M%S')
        time_list.append(time_yoink_dt)
        
    time_yoink_dt_array = np.array(time_list)
    time_yoink_dt_convert = time_yoink_dt_array.astype('datetime64[s]')
    time_yoink_dt_series = pd.Series(time_yoink_dt_convert)
    
    nearest_tobac_index = []
    for time in time_yoink_dt_convert:
        tobac_index = find_nearest(morton_tobac_times_datetime.data, time)
        nearest_tobac_index.append(tobac_index)
        
    tobac_times_closest = morton_tobac_times_datetime[nearest_tobac_index]
    series_tobac_times = pd.Series(tobac_times_closest)
    series_tobac_indeces = pd.Series(nearest_tobac_index)
    
    tobac_lats_closest = morton_tobac_lats[nearest_tobac_index]
    series_tobac_lats = pd.Series(tobac_lats_closest)
    tobac_lons_closest = morton_tobac_lons[nearest_tobac_index]
    series_tobac_lons = pd.Series(tobac_lons_closest)

    return time_yoink_dt_series, series_tobac_times, series_tobac_indeces, series_tobac_lats, series_tobac_lons

In [49]:
def vad_df(netcdf_list, gps_file, radar_array, velmean_array, velvar_array, bearmean_array,
           latmean_array, latvar_array, lonmean_array, lonvar_array, time_yoink_dt_series,
          series_tobac_times, series_tobac_indeces, series_tobac_lats, series_tobac_lons):
    '''
    This function uses alex's vehicle_correction_vad to create the below fields from the input
    radar object. These fields and the tobac information are all put into one pandas DataFrame.
    '''
    for file in netcdf_list:
        print(file)
        try:
            gps_file['ddmmyy'] = gps_file['ddmmyy'].astype(str)
            gps_file['hhmmss[UTC]'] = gps_file['hhmmss[UTC]'].astype(str)
            read = pyart.io.read(file)
            all_vehicle_correction = vehicle_correction_vad(read, gps_file)
            if all_vehicle_correction == None:
                print('broken')
                continue
            radar_array.append(all_vehicle_correction[0])
            velmean_array.append(all_vehicle_correction[1])
            velvar_array.append(all_vehicle_correction[2])
            bearmean_array.append(all_vehicle_correction[3])
            latmean_array.append(all_vehicle_correction[5])
            latvar_array.append(all_vehicle_correction[6])
            lonmean_array.append(all_vehicle_correction[7])
            lonvar_array.append(all_vehicle_correction[8])
        except Keyerror:
            pass

    radar_column = pd.Series(radar_array)
    velmean_column = pd.Series(velmean_array)
    velvar_column = pd.Series(velvar_array)
    bearmean_column = pd.Series(bearmean_array)
    latmean_column = pd.Series(latmean_array)
    latvar_column = pd.Series(latvar_array)
    lonmean_column = pd.Series(lonmean_array)
    lonvar_column = pd.Series(lonvar_array)

    distance_from_storm = measure(series_tobac_lats, series_tobac_lons, 
                                  latmean_column, lonmean_column)
    distance_from_storm_column = pd.Series(distance_from_storm)

    df = pd.DataFrame(pd.concat([time_yoink_dt_series,radar_column, velmean_column, velvar_column, 
                                 bearmean_column, latmean_column, latvar_column, lonmean_column, 
                                 lonvar_column, series_tobac_times, series_tobac_indeces, 
                                 series_tobac_lats, series_tobac_lons, distance_from_storm_column], axis = 1))
    
    df.rename(columns={0: 'Datetime', 1: 'Radar', 2: 'Velmean', 3: 'Velvar', 4: 'Bearmean', 5: 'Latmean', 
                            6: 'Latvar', 7: 'Lonmean', 8: 'Lonvar', 9: 'tobac_times',
                            10: 'tobac_indeces', 11: 'tobac_lats', 12: 'tobac_lons', 
                           13: 'Distance_From_Storm [km]'}, inplace  = True)

    return df

# Testing...

In [50]:
ka1_vads = sorted(glob.glob('/Users/juliabman/Desktop/vads/vads_radar_objects/ka1/05232022/*.nc'))
ka2_vads = sorted(glob.glob('/Users/juliabman/Desktop/vads/vads_radar_objects/ka2/05232022/*.nc'))
ka1gps = pd.read_csv('/Users/juliabman/Desktop/research2024/GPS_Ka1_20220523.txt')
ka2gps = pd.read_csv('/Users/juliabman/Desktop/research2024/GPS_Ka2_20220523.txt')
tobac_file = '/Users/juliabman/Desktop/research2024/tobac_Save/Track.nc'

In [51]:
np.shape(ka1_vads)

(82,)

In [52]:
np.shape(ka2_vads)

(163,)

In [53]:
cell_lats, cell_lons, cell_indeces, cell_times = get_storm_tobac(tobac_file, 29)

/var/folders/4f/9s_4__q16tx5t8vmww4mv03h0000gn/T/ipykernel_45591/2076589757.py:19: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  morton_tobac_times_datetime = morton_tobac_times.astype('datetime64[s]')


In [54]:
np.shape(cell_times)

(56,)

In [55]:
vad_times, morton_times, morton_indeces, morton_lats, morton_lons = nearest_tobac_time(ka1_vads, 
                                                                                           cell_lats, 
                                                                                           cell_lons, 
                                                                                           cell_times)

In [56]:
vad_times

0    2022-05-23 22:21:19
1    2022-05-23 22:22:19
2    2022-05-23 22:23:21
3    2022-05-23 22:23:51
4    2022-05-23 22:25:51
             ...        
77   2022-05-24 00:00:10
78   2022-05-24 00:00:39
79   2022-05-24 00:01:10
80   2022-05-24 00:01:39
81   2022-05-24 00:03:09
Length: 82, dtype: datetime64[s]

In [57]:
morton_times

0    2022-05-23 22:21:17
1    2022-05-23 22:21:17
2    2022-05-23 22:21:17
3    2022-05-23 22:25:49
4    2022-05-23 22:25:49
             ...        
77   2022-05-23 23:58:17
78   2022-05-23 23:58:17
79   2022-05-24 00:03:14
80   2022-05-24 00:03:14
81   2022-05-24 00:03:14
Length: 82, dtype: datetime64[ns]

In [58]:
morton_indeces

0      0
1      0
2      0
3      1
4      1
      ..
77    17
78    17
79    18
80    18
81    18
Length: 82, dtype: int64

In [59]:
morton_lats

0     33.725464
1     33.725464
2     33.725464
3     33.744333
4     33.744333
        ...    
77    33.903952
78    33.903952
79    33.900867
80    33.900867
81    33.900867
Length: 82, dtype: float64

In [60]:
morton_lons

0    -103.115808
1    -103.115808
2    -103.115808
3    -103.101291
4    -103.101291
         ...    
77   -102.746264
78   -102.746264
79   -102.732445
80   -102.732445
81   -102.732445
Length: 82, dtype: float64

In [61]:
radar_array = []
velmean_array = []
velvar_array = []
bearmean_array = []
latmean_array = []
latvar_array = []
lonmean_array = []
lonvar_array = []
ka1_df = vad_df(ka1_vads, ka1gps, radar_array, velmean_array, velvar_array, bearmean_array,
           latmean_array, latvar_array, lonmean_array, lonvar_array, vad_times,
          morton_times, morton_indeces, morton_lats, morton_lons)

/Users/juliabman/Desktop/vads/vads_radar_objects/ka1/05232022/dealiased_terminal_corrected_20220523222119.nc
/Users/juliabman/Desktop/vads/vads_radar_objects/ka1/05232022/dealiased_terminal_corrected_20220523222219.nc
/Users/juliabman/Desktop/vads/vads_radar_objects/ka1/05232022/dealiased_terminal_corrected_20220523222321.nc
/Users/juliabman/Desktop/vads/vads_radar_objects/ka1/05232022/dealiased_terminal_corrected_20220523222351.nc
/Users/juliabman/Desktop/vads/vads_radar_objects/ka1/05232022/dealiased_terminal_corrected_20220523222551.nc
/Users/juliabman/Desktop/vads/vads_radar_objects/ka1/05232022/dealiased_terminal_corrected_20220523222620.nc
/Users/juliabman/Desktop/vads/vads_radar_objects/ka1/05232022/dealiased_terminal_corrected_20220523222649.nc
/Users/juliabman/Desktop/vads/vads_radar_objects/ka1/05232022/dealiased_terminal_corrected_20220523222718.nc
/Users/juliabman/Desktop/vads/vads_radar_objects/ka1/05232022/dealiased_terminal_corrected_20220523222747.nc
/Users/juliabman/De

In [62]:
ka1_df

,Datetime,Radar,Velmean,Velvar,Bearmean,Latmean,Latvar,Lonmean,Lonvar,tobac_times,tobac_indeces,tobac_lats,tobac_lons,Distance_From_Storm [km]
0,2022-05-23 22:21:19,<pyart.core.radar.Radar object at 0x328cb5790>,62.339348,0.493868,270.572609,33.873966,1.609869e-09,-102.828631,2.176506e-05,2022-05-23 22:21:17,0,33.725464,-103.115808,31.288944
1,2022-05-23 22:22:19,<pyart.core.radar.Radar object at 0x30fc22520>,59.926889,2.204317,270.554889,33.874117,1.224962e-09,-102.848543,1.928859e-05,2022-05-23 22:21:17,0,33.725464,-103.115808,29.750265
2,2022-05-23 22:23:21,<pyart.core.radar.Radar object at 0x107acfe80>,53.586200,0.585208,265.355200,33.872104,2.886377e-07,-102.875865,7.335083e-05,2022-05-23 22:21:17,0,33.725464,-103.115808,27.552471
3,2022-05-23 22:23:51,<pyart.core.radar.Radar object at 0x3068a5040>,53.712037,0.517420,270.823704,33.871856,3.721434e-09,-102.877905,2.208765e-05,2022-05-23 22:25:49,1,33.744333,-103.101291,25.068904
4,2022-05-23 22:25:51,<pyart.core.radar.Radar object at 0x328bf0d60>,0.009848,0.002146,179.230000,33.871881,1.792075e-12,-102.898984,2.900816e-11,2022-05-23 22:25:49,1,33.744333,-103.101291,23.489548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,2022-05-24 00:00:10,<pyart.core.radar.Radar object at 0x30fca7e50>,5.490640,173.986013,239.566352,33.792004,3.526687e-05,-102.644818,1.318757e-06,2022-05-23 23:58:17,17,33.903952,-102.746264,15.597101
78,2022-05-24 00:00:39,<pyart.core.radar.Radar object at 0x30fc8c100>,36.328511,10.928426,1.018723,33.772682,5.268943e-06,-102.642528,2.740749e-09,2022-05-23 23:58:17,17,33.903952,-102.746264,17.479725
79,2022-05-24 00:01:10,<pyart.core.radar.Radar object at 0x30fca7cd0>,3.973097,137.133658,248.861781,33.793175,1.024336e-05,-102.644966,1.007866e-06,2022-05-24 00:03:14,18,33.900867,-102.732445,14.461335
80,2022-05-24 00:01:39,<pyart.core.radar.Radar object at 0x30fca7790>,43.196667,15.037367,38.183750,33.784412,8.276087e-06,-102.642297,2.191911e-09,2022-05-24 00:03:14,18,33.900867,-102.732445,15.412055


In [63]:
ka1_df.to_csv(path_or_buf = '/Users/juliabman/Desktop/vads/vad_dfs/ka1/05232022/ka1_vads_df')

In [ ]:
ka2_df.to_csv(path_or_buf = '/Users/juliabman/Desktop/vads/vad_dfs/ka2/05232022/ka2_vads_df')